In [1]:
!pip install torch numpy datasets scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install datasets

In [6]:
import torch
import json
import numpy as np
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from sklearn.preprocessing import StandardScaler, LabelEncoder
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from tqdm import tqdm

class DurakDataset(Dataset):
    def __init__(self, data, max_hand_size=6):
        self.max_hand_size = max_hand_size
        self.scaler = StandardScaler()
        self.label_encoder = LabelEncoder()

        # Собираем все уникальные карты
        all_cards = set()
        valid_data = []

        for game in data:
            try:
                snapshot = game['snapshot']
                if isinstance(snapshot, str):
                    snapshot = json.loads(snapshot)

                if 'players' in snapshot and len(snapshot['players']) == 2:
                    for player in snapshot['players']:
                        if 'hand' in player:
                            all_cards.update(player['hand'])
                    valid_data.append(snapshot)
            except (json.JSONDecodeError, KeyError) as e:
                continue

        if not valid_data:
            raise ValueError("No valid games found in dataset")

        # Создаем кодировщик карт
        self.card_encoder = {card: idx+1 for idx, card in enumerate(all_cards)}
        self.card_encoder['PAD'] = 0

        # Подготовка данных
        features = []
        labels = []

        for snapshot in valid_data:
            trump_suit = snapshot['trump'][-1] if 'trump' in snapshot else ''
            game_type = snapshot['game_rules']['game_type'] if 'game_rules' in snapshot else 0

            for player in snapshot['players']:
                # Кодируем карты в руке
                hand = player.get('hand', [])[:self.max_hand_size]
                hand_encoded = [self.card_encoder.get(card, 0) for card in hand]
                hand_encoded += [0] * (self.max_hand_size - len(hand))

                # Дополнительные признаки
                trump_cards = sum(1 for card in hand if card[-1] == trump_suit)
                features.append([
                    *hand_encoded,
                    game_type,
                    trump_cards,
                    len(hand),
                    len(snapshot.get('deck', [])),
                    len(snapshot.get('table', [])),
                    int(trump_suit == 'S'),
                    int(trump_suit == 'C'),
                    int(trump_suit == 'D'),
                    int(trump_suit == 'H')
                ])
                labels.append(player.get('state', 'unknown'))

        self.features = self.scaler.fit_transform(np.array(features, dtype=np.float32))
        self.labels = self.label_encoder.fit_transform(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return (
            torch.FloatTensor(self.features[idx]),
            torch.LongTensor([self.labels[idx]])
        )

class DurakModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

def train_model():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Загрузка данных
    try:
        dataset = load_dataset("neuronetties/durak")
        train_data = list(dataset['train'])  # Преобразуем в список для надежности
        print(f"Loaded {len(train_data)} training examples")
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return

    try:
        train_dataset = DurakDataset(train_data[:1000])  # Берем первые 1000 примеров для теста
        print(f"Created dataset with {len(train_dataset)} samples")
    except Exception as e:
        print(f"Error creating dataset: {e}")
        return

    # Создаем DataLoader
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    # Инициализация модели
    model = DurakModel(
        input_size=train_dataset.features.shape[1],
        num_classes=len(train_dataset.label_encoder.classes_)
    ).to(device)

    optimizer = Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    # Обучение
    for epoch in range(15):
        model.train()
        total_loss = 0

        for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            x, y = x.to(device), y.squeeze().to(device)

            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

    # Сохранение модели
    torch.save({
      'model_state_dict': model.state_dict(),
      'scaler_mean': train_dataset.scaler.mean_,
      'scaler_var': train_dataset.scaler.var_,
      'card_encoder': train_dataset.card_encoder,
      'label_encoder_classes': train_dataset.label_encoder.classes_,
      'input_size': train_dataset.features.shape[1]
  }, 'durak_model.pt', _use_new_zipfile_serialization=True)

if __name__ == "__main__":
    train_model()

Using device: cuda
Loaded 292962 training examples
Created dataset with 2000 samples


Epoch 1: 100%|██████████| 63/63 [00:00<00:00, 82.63it/s]


Epoch 1, Loss: 1.4564


Epoch 2: 100%|██████████| 63/63 [00:00<00:00, 508.76it/s]


Epoch 2, Loss: 1.2431


...

Epoch 3, Loss: 1.2048


Epoch 4: 100%|██████████| 63/63 [00:00<00:00, 486.34it/s]


Epoch 4, Loss: 1.1784


Epoch 5: 100%|██████████| 63/63 [00:00<00:00, 514.42it/s]


Epoch 5, Loss: 1.1528


In [14]:
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

class DurakTester:
    def __init__(self, model_path='durak_model.pt'):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model, self.scaler = self._load_model_and_scaler(model_path)
        self.model.eval()

    def _load_model_and_scaler(self, path):
        # Добавляем безопасные глобальные переменные
        from torch.serialization import add_safe_globals
        import numpy as np
        add_safe_globals([np._core.multiarray._reconstruct])

        # Загружаем checkpoint
        checkpoint = torch.load(path, map_location=self.device, weights_only=False)

        # Инициализируем модель
        model = DurakModel(
            input_size=checkpoint['input_size'],
            num_classes=len(checkpoint['label_encoder_classes'])
        ).to(self.device)
        model.load_state_dict(checkpoint['model_state_dict'])

        # Безопасная инициализация StandardScaler
        scaler = StandardScaler()
        if 'scaler_mean' in checkpoint and 'scaler_var' in checkpoint:
            scaler.mean_ = checkpoint['scaler_mean']
            scaler.var_ = checkpoint['scaler_var']
            scaler.scale_ = np.sqrt(checkpoint['scaler_var'])

            # Заменяем нулевые значения в scale_ на 1, чтобы избежать деления на 0
            scaler.scale_[scaler.scale_ == 0] = 1.0
            scaler.n_samples_seen_ = len(checkpoint['scaler_mean'])

        return model, scaler

    def _safe_transform(self, features):
        """Безопасное преобразование признаков с проверкой"""
        if not hasattr(self.scaler, 'scale_'):
            return features

        # Проверяем на NaN и бесконечности
        if np.any(np.isnan(features)) or np.any(np.isinf(features)):
            return np.zeros_like(features)

        try:
            # Нормализуем с проверкой деления на ноль
            normalized = (features - self.scaler.mean_) / self.scaler.scale_
            return np.nan_to_num(normalized, nan=0.0, posinf=0.0, neginf=0.0)
        except:
            return np.zeros_like(features)

    def prepare_test_sample(self, game_state):
        """Подготовка тестового примера с проверками"""
        try:
            trump_suit = game_state.get('trump', '')[-1] if game_state.get('trump') else ''
            game_type = game_state.get('game_rules', {}).get('game_type', 0)

            players = game_state.get('players', [])
            if not players:
                return None

            player = players[0]
            hand = player.get('hand', [])[:6]

            # Кодируем карты
            hand_encoded = [self.card_encoder.get(card, 0) for card in hand]
            hand_encoded += [0] * (6 - len(hand))

            # Собираем признаки
            features = np.array([
                *hand_encoded,
                game_type,
                sum(1 for card in hand if card[-1] == trump_suit),
                len(hand),
                len(game_state.get('deck', [])),
                len(game_state.get('table', [])),
                int(trump_suit == 'S'),
                int(trump_suit == 'C'),
                int(trump_suit == 'D'),
                int(trump_suit == 'H')
            ], dtype=np.float32)

            return features
        except:
            return None

    def predict_action(self, game_state):
        """Предсказание с обработкой ошибок"""
        try:
            features = self.prepare_test_sample(game_state)
            if features is None:
                return "unknown"

            features = self._safe_transform(features.reshape(1, -1))
            features_tensor = torch.FloatTensor(features).to(self.device)

            with torch.no_grad():
                output = self.model(features_tensor)
                predicted_idx = torch.argmax(output).item()

            return self.label_encoder.inverse_transform([predicted_idx])[0]
        except:
            return "unknown"

# Пример использования
if __name__ == "__main__":
    tester = DurakTester()

    # Тестовый пример
    test_game = {
        "trump": "10H",
        "game_rules": {"game_type": 0},
        "deck": ["11S", "10D"],
        "table": [{"attack_card": {"card": "10S", "user_id": "player2"}}],
        "players": [
            {"id": "player1", "state": "defend", "hand": ["10H", "11C", "14H"]},
            {"id": "player2", "state": "attack", "hand": ["9S", "12C"]}
        ]
    }

    prediction = tester.predict_action(test_game)
    print(f"Предсказанное действие: {prediction}")

Предсказанное действие: unknown
